# Source: https://github.com/chriscarrollsmith/imfp

# Script to import commodities prices from IMF

In [3]:
import pandas as pd
import requests
import imfp

from useful_functions import *

In [4]:
# Check the available databases
databases = imfp.imf_databases()

#Get the database for commodity prices
databases[databases['description'].str.contains("Commodity")]

,database_id,description
294,PCTOT,Commodity Terms of Trade
305,PCPS,Primary Commodity Price System (PCPS)


In [5]:
# Fetch list of valid parameters and input codes for commodity price database
params = imfp.imf_parameters("PCPS")

# Get key names from the params object
params.keys()

dict_keys(['freq', 'ref_area', 'commodity', 'unit_measure'])

In [6]:
imfp.imf_parameter_defs("PCPS")

,parameter,description
0,freq,Frequency
1,ref_area,Geographical Areas
2,commodity,Indicator
3,unit_measure,Unit


In [7]:
# View the data frame of valid input codes for the frequency parameter
params['freq']

,input_code,description
0,A,Annual
1,M,Monthly
2,Q,Quarterly


In [8]:
params['commodity']

,input_code,description
0,PRAWM,"Primary Commodity Prices, Agr. Raw Material In..."
1,PAGRI,"Primary Commodity Prices, Agriculture"
2,PALLFNF,"Primary Commodity Prices, All index"
3,PEXGMETA,"Primary Commodity Prices, All Metals EX GOLD I..."
4,PALLMETA,"Primary Commodity Prices, All Metals Index"
...,...,...
106,PWOOLC,"Primary Commodity Prices, Wool, Coarse"
107,PWOOLF,"Primary Commodity Prices, Wool, Fine"
108,POILWTI,"Primary Commodity Prices, WTI Crude"
109,PZINC,"Primary Commodity Prices, Zinc"


In [9]:
# Fetch the 'freq' input code for annual frequency
selected_freq = list(
    params['freq']['input_code'][params['freq']['description'].str.contains("Monthly")]
)

# List of substrings to check for
commodities_codes = ["PCOFFOTM", #Coffee, Other Mild Arabica 
                   "PIORECR", #Iron Ore
                   "PMEAT", #PMEAT index
                   "POILBRE", #Brent Crude Oil
                   "PORANG", #Orange 
                   "PSOYB", #Soybeans 
                   "PSUGA" #Sugar index 
                   ]

# Fetch the 'commodity' input code for coal
selected_commodity = list(
    params['commodity']['input_code'][params['commodity']['input_code'].isin(commodities_codes)]
)

# Fetch the 'unit_measure' input code for index
selected_unit_measure = list(
    params['unit_measure']['input_code'][params['unit_measure']['description'].str.contains("Index")]
)

# Request data from the API
df_commo = imfp.imf_dataset(database_id = "PCPS",
         freq = selected_freq, commodity = selected_commodity,
         unit_measure = selected_unit_measure,
         start_year = 2001, end_year = 2023)

# Display the first few entries in the retrieved data frame
df_commo = df_commo[['time_period', 'commodity', 'obs_value']]
df_commo = df_commo.rename(columns={'time_period': 'Time', 'obs_value': 'value'})
df_commo['commodity'] = df_commo['commodity'].replace({
    'PCOFFOTM': 'Coffee',
    'PIORECR': 'Iron Ore',
    'PMEAT': 'Meat index',
    'POILBRE': 'Brent Crude Oil',
    'PORANG': 'Orange',
    'PSOYB': 'Soybeans',
    'PSUGA': 'Sugar'
})





In [10]:
# Pivot the data frame
df_rotated = df_commo.pivot(index='Time', columns='commodity', values='value')

# Adjust index date format
df_commodities = adjust_index_date_format(df_rotated)

# Save the data frame to a CSV file
df_commodities.to_csv('../data/df_commodities.csv')

